In [1]:
import pandas as pd
import numpy as np

import os

from sklearn.model_selection import train_test_split
from keras.models import load_model

import shap

c:\Users\Phine\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reading de los modelos.

# Ruta donde están los modelos.

model_CDM = load_model(filepath = "../Versión 3 - actual/modelos/datasetCDM_2023_04_17_19_35")

model_GRD = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRD_2023_04_17_20_53")

model_GRDT = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRDT_2023_04_17_11_29")

model_SEV = load_model(filepath = "../Versión 3 - actual/modelos/datasetSEV_2023_04_17_22_09")

In [2]:
x = pd.read_csv("../../../Datos/Procesados/x_values.csv", dtype = int)
y_cdm = pd.read_csv("../../../Datos/Procesados/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv("../../../Datos/Procesados/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv("../../../Datos/Procesados/y_dummies_sev.csv", dtype = int)

In [3]:
lst_y = [y_cdm, y_grdt, y_grd, y_sev]
lst_y_str = ['CDM', 'GRDT', 'GRD', 'SEV']
# lst_model = [model_CDM, model_GRDT, model_GRD, model_SEV]

In [7]:
def data_prep(i, size_sample):
    x_train, x_test, y_train, y_test = train_test_split(x, lst_y[i], train_size = size_sample, test_size = size_sample, random_state = 0, stratify = lst_y[i])
    
    base_path = '../../../Datos/Procesados/split'
    exists = os.path.exists(path=base_path)
    if not exists:
        os.makedirs(base_path)

    x_train.to_csv(f'{base_path}/x_train_{lst_y_str[i]}')
    x_test.to_csv(f'{base_path}/x_test_{lst_y_str[i]}')
    y_train.to_csv(f'{base_path}/y_train_{lst_y_str[i]}')
    y_test.to_csv(f'{base_path}/y_test_{lst_y_str[i]}')



In [6]:
def shap_function(x_train, x_test, model, i, path_, model_name, size_sample): 
    # Función para predecir las probabilidades
    def predict_proba(X):
        return model.predict(X, verbose = False)

    # Crear un objeto KernelExplainer de SHAP
    background_data = shap.sample(x_train, 100)  # Usar una muestra del conjunto de entrenamiento como datos de fondo
    explainer = shap.KernelExplainer(predict_proba, background_data)

    # Calcular los valores SHAP para el conjunto de datos de prueba
    shap_values = explainer.shap_values(x_test, nsamples=100)

    # Convertir la lista shap_values_grd en una matriz numpy
    shap_values_np = np.array(shap_values)

    # Aplanar la matriz shap_values_grd_np a 2 dimensiones
    shap_values_flat = np.reshape(shap_values_np, (shap_values_np.shape[0], -1))

    # Convertir la matriz aplanada en un DataFrame de pandas
    shap_values_df = pd.DataFrame(shap_values_flat)

    isExists = os.path.exists(path=path_)
    if not isExists:
        os.makedirs(path_)

    # Guardar el DataFrame en un archivo CSV
    shap_values_df.to_csv(f"{path_}/shap_values_{model_name}_datasize_{size_sample}.csv", index=False)

    # Guardar las dimensiones originales en un archivo de texto
    with open(f"{path_}/shap_values_{model_name}_dimensions.txt", "w") as f:
        f.write(",".join(map(str, shap_values_np.shape)))


In [8]:
size_sample = 0.0005

for i in range(len(lst_y)):
# model_name = lst_y_str[3]
    # path = f"SHAP Datos/{lst_y_str[3]}"
    data_prep(i, size_sample)
    # shap_function(x_train, x_test, lst_model[3], 3, path, model_name, size_sample)